In [9]:
#Dependencies
from dwave.optimization import Model
from dwave.system import LeapHybridNLSampler
import resources
import numpy as np

In [10]:
#Function to permute states in the forward direction
def permute_state_forwards(perm):
    pairs = [(perm[i], i) for i in range(len(perm))]
    sorted_pairs = pairs.sort()
    return [pair[1] for pair in pairs]

#Function to permute states in the backward direction
def permute_state_backwards(perm):
    pairs = [(perm[i], i) for i in range(len(perm))]
    sorted_pairs = pairs.sort()
    return [pair[1] for pair in pairs]

print(permute_state_forwards([1, 3, 0, 2]))
print(permute_state_backwards([2, 0, 3, 1]))

[2, 0, 3, 1]
[1, 3, 0, 2]


In [11]:
#Function to find optimized permutation
def optimize(distance_m, flow_m):
    #Number of facilities and locations
    n = len(distance_m)

    #Create model
    model = Model()

    #Set parameters
    rooms_per_floor = [8, 6, 5]
    alpha = model.constant(25)          #Distance exchange coefficient
    beta = model.constant(2)            #Flow exchange coefficient
    gamma = model.constant(200000)      #Height exchange coefficient
    # delta = model.constant(7)         #Incorrect placement penalty

    #Define flow and distance in terms of model variables
    flow = model.constant(flow_m)
    distance = model.constant(distance_m)

    #Define iterables in terms of model variables
    j = model.constant(permute_state_forwards(list(range(n))))
    k = model.constant(permute_state_forwards(list(range(n))))

    #Get height difference
    accumulated_rooms_per_floor = [0]
    for i in range(len(rooms_per_floor)):
        accumulated_rooms_per_floor.append(accumulated_rooms_per_floor[i] + rooms_per_floor[i])
    height_m = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            h_i = -1
            while accumulated_rooms_per_floor[h_i + 1] <= i:
                h_i += 1
            h_j = -1
            while accumulated_rooms_per_floor[h_j + 1] <= j:
                h_j += 1
            height_m[i][j] = abs(h_i - h_j)
    height = model.constant(height_m)

    #Define permutation ListVariable
    perm = model.list(n)

    #Define cost function
    c = (distance[j][k] * (alpha + flow[perm[j]][perm[k]]) + beta * (flow[perm[j]][perm[k]] ** 2) + gamma * height[j][k] ).sum()

    #Minimize cost function
    model.minimize(c)

    #Sample distribution
    sampler = LeapHybridNLSampler()
    results = sampler.sample(
        model,
        label='example2'
    )

    #Get non-permutated state
    route, = model.iter_decisions()
    state_non_permutated = route.state(0)

    #Get permutated state
    final_state = permute_state_backwards(state_non_permutated)

    #Get cost
    cost = 0
    for j in range(n):
        for k in range(j+1, n):
            cost += distance_m[j][k] * flow_m[final_state[j]][final_state[k]]

    return final_state, cost

In [12]:
#Example 1
dist_m = [[0,  10, 2, 4],
          [10, 0,  5, 7],
          [2,  5,  0, 6],
          [4,  7,  6, 0]]

flow_m = [[0, 1,   2,   4],
          [1, 0,   3.5, 3],
          [2, 3.5, 0,   1],
          [4, 3,   1,   0]]

print(optimize(dist_m, flow_m))

([2, 0, 3, 1], 81.0)


In [13]:
#Example 2
flow_m = [[0, 3, 1],
          [3, 0, 2],
          [1, 2, 0]]

dist_m = [[0, 1, 3],
          [1, 0, 2],
          [3, 2, 0]]

print(optimize(dist_m, flow_m))

([0, 1, 2], 10)


In [14]:
#Example 3 (with real hospital data)
print(optimize(resources.dist_m, resources.flow_m))

([17, 16, 1, 12, 5, 2, 6, 14, 7, 10, 15, 8, 13, 18, 4, 3, 0, 9, 11], 24445895)


In [15]:
#Time Evolution
total_time = 12 #1 year
flow = resources.flow_m
dist = resources.dist_m
n = len(flow)

for t in range(total_time):
    #Find optimal solution for this month
    optimized_sequence, optimized_cost = optimize(dist, flow)
    print(f"Optimal solution in month {t}")
    print(f"    Optimal sequence of faclity placements: {optimized_sequence}")
    print(f"    Optimal cost of the sequence: {optimized_cost}\n")

    #Vary flow rates over time by scaling each one by a random number sampled from normal distribution ranging from scaling factors of -10% to +10%
    variation = np.zeros((n, n))
    for j in range(n):
        for k in range(j+1, n):
            while variation[j][k] < .9 or variation[j][k] > 1.1:
                variation[j][k] = np.random.normal(1, .05)
            #Scale each flow by corresponding variation factor
            flow[j][k] = flow[j][k] * variation[j][k]
            flow[k][j] = flow[j][k]

Optimal solution in month 0
    Optimal sequence of faclity placements: [17, 16, 4, 2, 15, 10, 14, 13, 5, 9, 7, 0, 11, 18, 3, 12, 1, 6, 8]
    Optimal cost of the sequence: 31709473

Optimal solution in month 1
    Optimal sequence of faclity placements: [17, 16, 9, 13, 10, 5, 7, 14, 1, 6, 3, 12, 4, 18, 15, 8, 11, 0, 2]
    Optimal cost of the sequence: 27066528.05808285

Optimal solution in month 2
    Optimal sequence of faclity placements: [17, 16, 12, 2, 15, 9, 6, 7, 5, 0, 3, 8, 4, 18, 11, 13, 14, 1, 10]
    Optimal cost of the sequence: 38058576.20264058

Optimal solution in month 3
    Optimal sequence of faclity placements: [17, 16, 2, 0, 4, 10, 13, 3, 12, 5, 11, 6, 1, 18, 15, 14, 9, 7, 8]
    Optimal cost of the sequence: 17164571.605204597

Optimal solution in month 4
    Optimal sequence of faclity placements: [17, 10, 4, 12, 6, 9, 0, 13, 8, 7, 5, 3, 1, 18, 2, 11, 15, 16, 14]
    Optimal cost of the sequence: 27246652.624653365

Optimal solution in month 5
    Optimal sequenc